<a href="https://colab.research.google.com/github/abubakrsiddq/ImageDehazing/blob/main/models/novel/a_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow_addons
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import random
from PIL import Image
import time
import tensorflow_addons as tfa
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.losses import mean_squared_error
from tensorflow.keras.optimizers import Adam
import keras

     |████████████████████████████████| 706kB 9.9MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def load_image(img_path):
    img = tf.io.read_file(img_path)
    #img = tf.io.decode_jpeg(img, channels = 3)
    img = tf.io.decode_png(img, channels = 3)
    img = tf.image.resize(img, size = (352, 1216), antialias = True)
    img = img / 255.0
    return img

In [ ]:
def load_airlight(img_path):
  air=img_path.split("_")[-1][:-4];
  return air

In [ ]:
def display_img(model, hazy_img, orig_img):
    pred = model(hazy_img)
    plt.figure(figsize = (15,15))
    
    display_list = [hazy_img[0], orig_img[0], pred[0]]
    title = ['Hazy Image', 'Ground Truth', 'Dehazed Image']
    
    for i in range(3):
        plt.subplot(1, 3, i+1)
        plt.title(title[i])
        plt.imshow(display_list[i])
        plt.axis('off')
        
    plt.show()

In [ ]:
def data_path(hazy_img_path,percentage=0.8,seed_val=101):
    
    random.seed(seed_val)
    train_img = []
    val_img = []
    train_air= []
    val_air= []
    
    
    #hazy_img = glob.glob(hazy_img_path + '/*.jpg')
    hazy_img = glob.glob(hazy_img_path + '/*.png')
    n = len(hazy_img)
    n1=percentage*n;

    random.shuffle(hazy_img)
    n=n1;
    train_keys = hazy_img[:int(0.9*n)]        #90% data for train, 10% for test
    val_keys = hazy_img[int(0.9*n):int(1.0*n)]
    #hazy_img=hazy_img[:n]
    split_dict = {}
    for key in train_keys:
        split_dict[key] = 'train'
    for key in val_keys:
        split_dict[key] = 'val'
        
    
    for img in hazy_img:
        air=img.split("_")[-1][:-4];
  
        try:
          if (split_dict[img] == 'train'):
              train_air.append(float(air))
              train_img.append(img)
          else:
              val_air.append(float(air))
              val_img.append(img)
        except KeyError:
          pass
    return train_img,train_air, val_img,val_air

In [ ]:
def dataloader(train_data,train_res, val_data,val_res, batch_size):
    print(len(train_data))
    train_data_haze = tf.data.Dataset.from_tensor_slices([img for img in train_data]).map(lambda x: load_image(x))
    train_data_airlight = tf.data.Dataset.from_tensor_slices([a for a in train_res])
    train = tf.data.Dataset.zip((train_data_haze,train_data_airlight)).shuffle(100).batch(batch_size)

    val_data_haze = tf.data.Dataset.from_tensor_slices([img for img in val_data]).map(lambda x: load_image(x))
    val_data_airlight = tf.data.Dataset.from_tensor_slices([a for a in val_res])
    val = tf.data.Dataset.zip((val_data_haze,val_data_airlight)).shuffle(100).batch(batch_size)

    

    

    return train, val

In [ ]:
def build_model(input_shape):
    """Generates CNN model
    :param input_shape (tuple): Shape of input set
    :return model: CNN model
    """

    # build network topology
    model = keras.Sequential()

    # 1st conv layer
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # 2nd conv layer
    model.add(keras.layers.Conv2D(16, (3, 3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # 3rd conv layer
    model.add(keras.layers.Conv2D(8, (2, 2), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # flatten output and feed it into dense layer
    model.add(keras.layers.Flatten())
    
    # 1st dense layer
    keras.layers.Dense(512, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dropout(0.6),

    # 2nd dense layer
    keras.layers.Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dropout(0.5),

    # 3rd dense layer
    keras.layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.Dropout(0.2),


    # output layer
    keras.layers.Dense(1, activation='relu')

    return model

In [ ]:
# Hyperparameters
epochs = 20
batch_size = 32
k_init = tf.keras.initializers.random_normal(stddev=0.008, seed = 101)      
regularizer = tf.keras.regularizers.L2(1e-4)
b_init = tf.constant_initializer()

train_data,train_res, val_data,val_res = data_path(hazy_img_path = '/content/drive/MyDrive/kitti/hazy')
train, val = dataloader(train_data,train_res, val_data,val_res, batch_size)

optimizer = Adam(learning_rate = 1e-5)
net = build_model([352, 1216, 3])
train_loss_tracker = tf.keras.metrics.MeanSquaredError(name = "train loss")
val_loss_tracker = tf.keras.metrics.MeanSquaredError(name = "val loss")

4320


In [ ]:
def train_model(epochs, train, val, net, train_loss_tracker, val_loss_tracker, optimizer):
    
    for epoch in range(epochs):
        
        print("\nStart of epoch %d" % (epoch,), end=' ')
        start_time_epoch = time.time()
        start_time_step = time.time()

        for step, (train_batch_haze, train_res) in enumerate(train):

            with tf.GradientTape() as tape:

                train_logits = net(train_batch_haze, training = True)
                #print(train_logits.shape)
                train_res=np.expand_dims(train_res,axis=-1)
                loss = mean_squared_error(train_res, train_logits)

            grads = tape.gradient(loss, net.trainable_weights)
            optimizer.apply_gradients(zip(grads, net.trainable_weights))
            
            train_loss_tracker.update_state(train_res, train_logits)
            if step == 0:
                print('[', end='')
            if step % 64 == 0:
                print('=', end='')
        
        print(']', end='')
        print('  -  ', end='')
        print('Training Loss: %.4f' % (train_loss_tracker.result()), end='')
        
        for step, (val_batch_haze, val_res) in enumerate(val):
            val_logits = net(val_batch_haze, training = False)
            val_res=np.expand_dims(val_res,axis=-1)
            #print(val_logits.shape)
            #print("---------------------")
            val_loss_tracker.update_state(val_res, val_logits)
            #if step % 64 ==0:
                #display_img(net, val_batch_haze, val_batch_orig)
        print('  -  ', end='')
        print('Validation Loss: %.4f' % (val_loss_tracker.result()), end='')
        print('  -  ', end=' ')
        print("Time taken: %.2fs" % (time.time() - start_time_epoch))
        
        train_loss_tracker.reset_states()
        val_loss_tracker.reset_states()

In [ ]:
%%time
train_model(10, train, val, net, train_loss_tracker, val_loss_tracker, optimizer)


Start of epoch 0 [===]  -  Training Loss: 1.6729  -  Validation Loss: 1.1730  -   Time taken: 2144.46s

Start of epoch 1 [===]  -  Training Loss: 1.6527  -  Validation Loss: 1.1618  -   Time taken: 173.54s

Start of epoch 2 [===]  -  Training Loss: 1.6372  -  Validation Loss: 1.3010  -   Time taken: 171.64s

Start of epoch 3 [===]  -  Training Loss: 1.6282  -  Validation Loss: 1.4839  -   Time taken: 171.21s

Start of epoch 4 [===]  -  Training Loss: 1.6204  -  Validation Loss: 1.5830  -   Time taken: 171.18s

Start of epoch 5 [===]  -  Training Loss: 1.6130  -  Validation Loss: 1.6045  -   Time taken: 171.06s

Start of epoch 6 [===]  -  Training Loss: 1.6061  -  Validation Loss: 1.6167  -   Time taken: 171.00s

Start of epoch 7 [===]  -  Training Loss: 1.5991  -  Validation Loss: 1.6404  -   Time taken: 171.02s

Start of epoch 8 [===]  -  Training Loss: 1.5922  -  Validation Loss: 1.6313  -   Time taken: 170.75s

Start of epoch 9 [===]  -  Training Loss: 1.5851  -  Validation Loss: 1

In [ ]:
optimizer = Adam(learning_rate = 1e-3)

In [ ]:
%%time
train_model(20, train, val, net, train_loss_tracker, val_loss_tracker, optimizer)


Start of epoch 0 [===]  -  Training Loss: 1.0607  -  Validation Loss: 1.5361  -   Time taken: 172.84s

Start of epoch 1 [===]  -  Training Loss: 0.3803  -  Validation Loss: 0.6960  -   Time taken: 172.45s

Start of epoch 2 [===]  -  Training Loss: 0.2599  -  Validation Loss: 0.3418  -   Time taken: 172.06s

Start of epoch 3 [===]  -  Training Loss: 0.1744  -  Validation Loss: 0.1790  -   Time taken: 171.66s

Start of epoch 4 [===]  -  Training Loss: 0.1162  -  Validation Loss: 0.1012  -   Time taken: 172.12s

Start of epoch 5 [===]  -  Training Loss: 0.0785  -  Validation Loss: 0.0634  -   Time taken: 172.28s

Start of epoch 6 [===]  -  Training Loss: 0.0553  -  Validation Loss: 0.0445  -   Time taken: 171.74s

Start of epoch 7 [===]  -  Training Loss: 0.0420  -  Validation Loss: 0.0349  -   Time taken: 171.90s

Start of epoch 8 [===]  -  Training Loss: 0.0349  -  Validation Loss: 0.0302  -   Time taken: 171.41s

Start of epoch 9 [===]  -  Training Loss: 0.0313  -  Validation Loss: 0.

In [ ]:
optimizer = Adam(learning_rate = 1e-7)

In [ ]:
%%time
train_model(20, train, val, net, train_loss_tracker, val_loss_tracker, optimizer)


Start of epoch 0 [===]  -  Training Loss: 0.0286  -  Validation Loss: 0.0271  -   Time taken: 172.55s

Start of epoch 1 [===]  -  Training Loss: 0.0286  -  Validation Loss: 0.0271  -   Time taken: 171.65s

Start of epoch 2 [===]  -  Training Loss: 0.0286  -  Validation Loss: 0.0271  -   Time taken: 171.25s

Start of epoch 3 [===]  -  Training Loss: 0.0286  -  Validation Loss: 0.0271  -   Time taken: 171.01s

Start of epoch 4 [===]  -  Training Loss: 0.0286  -  Validation Loss: 0.0271  -   Time taken: 172.48s

Start of epoch 5 [===]  -  Training Loss: 0.0286  -  Validation Loss: 0.0271  -   Time taken: 171.11s

Start of epoch 6 [===]  -  Training Loss: 0.0286  -  Validation Loss: 0.0271  -   Time taken: 171.05s

Start of epoch 7 [===]  -  Training Loss: 0.0286  -  Validation Loss: 0.0271  -   Time taken: 170.93s

Start of epoch 8 [===]  -  Training Loss: 0.0286  -  Validation Loss: 0.0271  -   Time taken: 172.29s

Start of epoch 9 [===]  -  Training Loss: 0.0286  -  Validation Loss: 0.

In [ ]:
net.save('/content/drive/MyDrive/nets/prelim_model/a_net/0.0271')
net.save_weights('/content/drive/MyDrive/nets/prelim_model/a_net/weights/0.0271')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/nets/prelim_model/a_net/0.0271/assets
